In [1]:
%reset -f

In [2]:
import numpy as np
import pandas as pd
import datetime
import time
import re
import requests
from itertools import islice

api_key = pd.read_json('fmp_api_key.json').iloc[0].to_list()[0]
data_path = '/Users/ayemyatwinshwe/repos/data/trading_data/'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15'}

In [3]:
df_earning_calls = pd.read_parquet(data_path + 'earning_calls')
df_earning_calls['date'] = df_earning_calls['date'].dt.strftime('%Y-%m-%d')
df_earning_calls['date'] = pd.to_datetime(df_earning_calls['date'])
df_earning_calls.set_index(['symbol'], inplace=True)
df_earning_calls['prev_day'] = df_earning_calls['date'].apply(lambda x: x-pd.offsets.BDay(1))
df_earning_calls['next_7day'] = df_earning_calls['date'].apply(lambda x: x+pd.offsets.BDay(7))
df_earning_calls.head()

,quarter,year,date,content,prev_day,next_7day
symbol,,,,,,
A,1,2019,2019-02-21,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04
A,2,2019,2019-05-15,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24
A,3,2019,2019-08-15,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26
A,4,2019,2019-11-26,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05
A,1,2020,2020-02-19,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28


In [4]:
tickers = np.unique(df_earning_calls.index.get_level_values(0))

In [5]:
df_eod = pd.read_parquet(data_path + 'eod')
df_eod = df_eod.loc[tickers]
df_adjClose = df_eod[['adjClose']]
df_adjClose.reset_index(inplace=True)
df_adjClose.set_index(['symbol'], inplace=True)
df_adjClose.head()

,date,adjClose
symbol,,
A,2018-05-11,67.639832
A,2018-05-14,67.406090
A,2018-05-15,60.870972
A,2018-05-16,60.325573
A,2018-05-17,61.932568


In [6]:
# set prev_day as  date to merge with df_adjClose
df_earning_calls['date'] = df_earning_calls['prev_day']
df_earning_calls.head()

,quarter,year,date,content,prev_day,next_7day
symbol,,,,,,
A,1,2019,2019-02-20,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04
A,2,2019,2019-05-14,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24
A,3,2019,2019-08-14,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26
A,4,2019,2019-11-25,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05
A,1,2020,2020-02-18,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28


In [7]:
df_earning_calls = pd.merge(df_earning_calls, df_adjClose, on=['symbol','date'], how='left')
df_earning_calls.rename(columns={'adjClose': 'prev_adjClose'}, inplace=True)

In [8]:
df_earning_calls

,quarter,year,date,content,prev_day,next_7day,prev_adjClose
symbol,,,,,,,
A,1,2019,2019-02-20,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04,77.049599
A,2,2019,2019-05-14,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24,74.701042
A,3,2019,2019-08-14,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26,64.602371
A,4,2019,2019-11-25,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05,79.221558
A,1,2020,2020-02-18,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28,83.869682
...,...,...,...,...,...,...,...
ZYXI,4,2019,2020-02-27,"Operator: Good afternoon, and welcome to the Z...",2020-02-27,2020-03-10,11.920000
ZYXI,1,2020,2020-04-28,Operator: Good day and welcome to the Zynex Fi...,2020-04-28,2020-05-08,15.540000
ZYXI,2,2020,2020-07-27,"Operator: Good day, and welcome to the Zynex S...",2020-07-27,2020-08-06,19.510000


In [9]:
# set next_7day as  date to merge with df_adjClose to get next 7th day eod prices
df_earning_calls['date'] = df_earning_calls['next_7day']
df_earning_calls.head()

,quarter,year,date,content,prev_day,next_7day,prev_adjClose
symbol,,,,,,,
A,1,2019,2019-03-04,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04,77.049599
A,2,2019,2019-05-24,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24,74.701042
A,3,2019,2019-08-26,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26,64.602371
A,4,2019,2019-12-05,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05,79.221558
A,1,2020,2020-02-28,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28,83.869682


In [10]:
df_earning_calls = pd.merge(df_earning_calls, df_adjClose, on=['symbol','date'], how='left')
df_earning_calls.rename(columns={'adjClose': 'next_adjClose'}, inplace=True)

In [11]:
df_earning_calls

,quarter,year,date,content,prev_day,next_7day,prev_adjClose,next_adjClose
symbol,,,,,,,,
A,1,2019,2019-03-04,"Operator: Good day, ladies and gentlemen, and ...",2019-02-20,2019-03-04,77.049599,78.589417
A,2,2019,2019-05-24,"Operator: Good day, ladies and gentlemen. And ...",2019-05-14,2019-05-24,74.701042,67.182785
A,3,2019,2019-08-26,"Operator: Good afternoon, and welcome to the A...",2019-08-14,2019-08-26,64.602371,68.591385
A,4,2019,2019-12-05,"Operator: Good afternoon, and welcome to the A...",2019-11-25,2019-12-05,79.221558,80.475121
A,1,2020,2020-02-28,Operator: Good afternoon and welcome to the Ag...,2020-02-18,2020-02-28,83.869682,76.233475
...,...,...,...,...,...,...,...,...
ZYXI,4,2019,2020-03-10,"Operator: Good afternoon, and welcome to the Z...",2020-02-27,2020-03-10,11.920000,11.900000
ZYXI,1,2020,2020-05-08,Operator: Good day and welcome to the Zynex Fi...,2020-04-28,2020-05-08,15.540000,18.530000
ZYXI,2,2020,2020-08-06,"Operator: Good day, and welcome to the Zynex S...",2020-07-27,2020-08-06,19.510000,16.820000


In [12]:
df_earning_calls.isnull().any()

quarter          False
year             False
date             False
content          False
prev_day         False
next_7day        False
prev_adjClose     True
next_adjClose     True
dtype: bool

In [13]:
df_earning_calls.dropna(inplace=True)

In [14]:
df_earning_calls.isnull().any()

quarter          False
year             False
date             False
content          False
prev_day         False
next_7day        False
prev_adjClose    False
next_adjClose    False
dtype: bool

In [15]:
df_earning_calls.to_parquet(data_path + 'earning_calls_1', engine='pyarrow', compression='gzip')